In [34]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import datetime
from natsort import natsorted
import math
import tqdm
import glob
from os.path import basename
import shutil

In [ ]:
# The features for song ID 1-1000 are longer than 45 seconds, we 

## Step 1: Getting song start time in seconds

In [2]:
meta_df = pd.read_csv('metadata/metadata_2013.csv')

def minutes_to_seconds(time):
    time_tuple = math.modf(time)
    seconds = time_tuple[1]*60 + time_tuple[0]*100
    return seconds

meta_df['start of the segment (min.sec)'] = meta_df['start of the segment (min.sec)'].apply(lambda x: minutes_to_seconds(x))
#Getting the start time in seconds

meta_df_processed = meta_df
# meta_df.to_csv('metadata_2013_start_time_in_seconds.csv') # You can export it as a csv file if you like

## Step 2: Getting the first 45s excerpt for features

In [3]:
#Removing colons from csv file
i=0

x_list = [] #list to hold the features for all audio (frame time columns removed)
audio_processed_list = []

#list for saveing the missing file names
y_exist_list = []
missing_audiofiles = []


for file in tqdm.tqdm(natsorted(glob.glob("./features/*.csv"))):
#     feature_file = ".\\acp\\search\\data\\Downloaded\\features\\4.csv"
    if int(basename(file)[:-4]) in meta_df.values[:,0]: #Check if the song id exist
        df_features = pd.read_csv(file, sep=';')
        df_features_noframe = df_features.iloc[:,1:].values #Is it true?
        x_list.append(df_features_noframe)
        audio_processed_list.append(basename(file))
#         print(basename(file)[:-4])
        start_time = meta_df_processed[meta_df_processed['song_id'] == int(basename(file)[:-4])]['start of the segment (min.sec)'].values[0] # getting the start time when the song_id is known
        start_time = round(start_time,1)
        start_index = df_features[df_features['frameTime'] == start_time].index[0]
        df_features_processed = df_features[start_index:start_index+88]
        df_features_processed.to_csv('./processed/audio_feat/'+basename(file),index=False, sep=';')
       
        df_yindex = meta_df[meta_df['song_id']==int(basename(file)[:-4])].index.tolist()
#         meta_df_processed = meta_df_processed.append(meta_df[df_yindex[0]:df_yindex[0]+1])
        y_exist_list.append(meta_df['song_id'][df_yindex[0]])
        i+=1
            
    else:
        print('missing file = {}'.format(basename(file)))
        missing_audiofiles.append(basename(file))
        continue
missing_annotations = set(meta_df['song_id'].tolist()).symmetric_difference(set(y_exist_list))
print("Missing files in audio = {}".format(missing_audiofiles))
print("Missing files in annotations = {}".format(missing_annotations))


 74%|███████▎  | 1326/1802 [00:56<00:17, 27.07it/s]

missing file = 1001.csv
missing file = 1002.csv
missing file = 1003.csv
missing file = 1004.csv
missing file = 1005.csv
missing file = 1006.csv
missing file = 1007.csv
missing file = 1008.csv
missing file = 1009.csv
missing file = 1010.csv
missing file = 1011.csv
missing file = 1012.csv
missing file = 1013.csv
missing file = 1014.csv
missing file = 1015.csv
missing file = 1016.csv
missing file = 1017.csv
missing file = 1018.csv
missing file = 1019.csv
missing file = 1020.csv
missing file = 1021.csv
missing file = 1022.csv
missing file = 1023.csv
missing file = 1024.csv
missing file = 1025.csv
missing file = 1026.csv
missing file = 1027.csv
missing file = 1028.csv
missing file = 1029.csv
missing file = 1030.csv
missing file = 1031.csv
missing file = 1032.csv
missing file = 1033.csv
missing file = 1034.csv
missing file = 1035.csv
missing file = 1036.csv
missing file = 1037.csv
missing file = 1038.csv
missing file = 1039.csv
missing file = 1040.csv
missing file = 1041.csv
missing file = 1

100%|██████████| 1802/1802 [00:57<00:00, 31.51it/s]

missing file = 1724.csv
missing file = 1725.csv
missing file = 1726.csv
missing file = 1727.csv
missing file = 1728.csv
missing file = 1729.csv
missing file = 1730.csv
missing file = 1731.csv
missing file = 1732.csv
missing file = 1733.csv
missing file = 1734.csv
missing file = 1735.csv
missing file = 1736.csv
missing file = 1737.csv
missing file = 1738.csv
missing file = 1739.csv
missing file = 1740.csv
missing file = 1741.csv
missing file = 1742.csv
missing file = 1743.csv
missing file = 1744.csv
missing file = 1745.csv
missing file = 1746.csv
missing file = 1747.csv
missing file = 1748.csv
missing file = 1749.csv
missing file = 1750.csv
missing file = 1751.csv
missing file = 1752.csv
missing file = 1753.csv
missing file = 1754.csv
missing file = 1755.csv
missing file = 1756.csv
missing file = 1757.csv
missing file = 1758.csv
missing file = 1759.csv
missing file = 1760.csv
missing file = 1761.csv
missing file = 1762.csv
missing file = 1763.csv
missing file = 1764.csv
missing file = 1

## Step 3: Moving song ID 1001-2000 to the Processed file

In [10]:
for file in tqdm.tqdm(natsorted(glob.glob("./features/*.csv"))):
    if int(basename(file)[:-4]) > 1000:
        shutil.move(file, './processed/audio_feat/'+basename(file))

100%|██████████| 1802/1802 [00:00<00:00, 51486.99it/s]


## Step 4: Loading the features as a numpy array

In [58]:
df_annotations = pd.read_csv('./annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv')

In [60]:
# The arousal and valence columns from the dataset is still incorrect as of 20 July 2020
# Swapping them back
df_annotations[[' valence_mean', ' arousal_mean']] = df_annotations[[' arousal_mean', ' valence_mean']] 
df_annotations[[' valence_std', ' arousal_std']] = df_annotations[[' arousal_std', ' valence_std']] 

In [31]:
#Removing colons from csv file
i=0
x_list = []
file_order = []
missingfile_order = []
y_df = pd.DataFrame(columns=['song_id', ' valence_mean', ' valence_std', ' arousal_mean',' arousal_std'])
for file in tqdm.tqdm(natsorted(glob.glob("./processed/audio_feat/*.csv"))):
#     feature_file = ".\\acp\\search\\data\\Downloaded\\features\\4.csv"
    if int(basename(file)[:-4]) in df_annotations.values[:,0]: #Check if the song id exist
        df_features = pd.read_csv(file, sep=';')
        df_features_noframe = df_features.iloc[:88,1:].values #Is it true?
        x_list.append(df_features_noframe)
        file_order.append(basename(file))

#         print(basename(file)[:-4])
        try:
            df_yindex = df_annotations[df_annotations['song_id']==int(basename(file)[:-4])].index.tolist()  
            y_df = y_df.append(df_annotations[df_yindex[0]:df_yindex[0]+1])
            i+=1
        except:
            print("error in = {}".format(basename(file)))
            
    else:
        missingfile_order.append(basename(file))
        continue
print("Missing files = {}".format(missingfile_order))

100%|██████████| 1802/1802 [00:43<00:00, 41.23it/s]

Missing files = ['2001.csv', '2002.csv', '2003.csv', '2004.csv', '2005.csv', '2006.csv', '2007.csv', '2008.csv', '2009.csv', '2010.csv', '2011.csv', '2012.csv', '2013.csv', '2014.csv', '2015.csv', '2016.csv', '2017.csv', '2018.csv', '2019.csv', '2020.csv', '2021.csv', '2022.csv', '2023.csv', '2024.csv', '2025.csv', '2026.csv', '2027.csv', '2028.csv', '2029.csv', '2030.csv', '2031.csv', '2032.csv', '2033.csv', '2034.csv', '2035.csv', '2036.csv', '2037.csv', '2038.csv', '2039.csv', '2040.csv', '2041.csv', '2042.csv', '2043.csv', '2044.csv', '2045.csv', '2046.csv', '2047.csv', '2048.csv', '2049.csv', '2050.csv', '2051.csv', '2052.csv', '2053.csv', '2054.csv', '2055.csv', '2056.csv', '2057.csv', '2058.csv']


In [66]:
np.save('X_DEAM', x_list)
np.save('y_valence_mean', y_df[' valence_mean'].values)
np.save('y_arousal_mean', y_df[' arousal_mean'].values)